In [4]:
!pip install --upgrade pip --user

In [7]:
!pip install nltk
!pip install pyenchant
!pip install pyPDF2
!pip install pillow
!pip install requests
!pip install beautifulsoup4
!pip install tqdm

In [8]:
import os
import PyPDF2

In [9]:
def pdf_to_text(pdf_file_path):
    text = ""
    # Открываем PDF файл
    with open(pdf_file_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        # # Создаем объект для записи текста в файл
        # with open(output_txt_file, 'w', encoding='utf-8') as txt_file:
        # Проходим по всем страницам PDF
        for page_num in range(len(pdf_reader.pages)):
            # Получаем текст со страницы
            page = pdf_reader.pages[page_num]
            text += page.extract_text()

            # # Записываем текст в файл
            # txt_file.write(text)

    print("PDF успешно сконвертирован в TXT.")

    return text

In [10]:
def process_articles(input_dir):
    # Создаем пустую строку для объединения текста из всех статей
    combined_text = ""

    # Проходим по всем файлам в директории
    for filename in os.listdir(input_dir):
        if filename.endswith(".pdf"):
            # Получаем путь к файлу
            file_path = os.path.join(input_dir, filename)

            # Преобразуем PDF в текст и добавляем его к общему тексту
            article_text = pdf_to_text(file_path)
            combined_text += article_text

    return combined_text

In [11]:
# Путь к директории с PDF файлами
input_dir = "articles"

# Обрабатываем все статьи
combined_text = process_articles(input_dir)

# Путь для сохранения объединенного текста
output_txt_file = "articles.txt"

# Сохраняем объединенный текст в файл
with open(output_txt_file, 'w', encoding='utf-8') as txt_file:
    txt_file.write(combined_text)

print("Все статьи успешно сконвертированы в один TXT файл.")

PDF успешно сконвертирован в TXT.
Все статьи успешно сконвертированы в один TXT файл.


In [12]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
import enchant
import re

In [18]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Daminov\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Daminov\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Daminov\AppData\Roaming\nltk_data...


True

In [19]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [20]:
def lemmatize_word(word, pos):
    if pos is None:
        return word
    else:
        lemma = nltk.WordNetLemmatizer().lemmatize(word, pos=pos)
        return lemma

In [21]:
def extract_unique_words(text):
    tokens = word_tokenize(text)
    tagged_words = nltk.pos_tag(tokens)
    unique_words = set()

    for word, tag in tagged_words:
        pos = get_wordnet_pos(tag)
        lemma = lemmatize_word(word.lower(), pos)
        unique_words.add(lemma)

    return unique_words

In [22]:
def filter_english_words(words):
    d = enchant.Dict("en_US")
    english_words = set()
    for word in words:
        # Проверяем, является ли слово английским и находится ли оно в словаре
        if re.match(r'^[a-zA-Z\-]+$', word) and d.check(word):
            english_words.add(word)
    return english_words

In [24]:
with open('articles.txt', 'r', encoding='utf-8') as file:
    text = file.read()

unique_words = extract_unique_words(text)
filtered_wors =  filter_english_words(unique_words)

with open('words.txt', 'w') as file:
    for word in filtered_wors:
        file.write(word + '\n')

print("Уникальные слова:")
print(len(filtered_wors))

Уникальные слова:
1069


In [25]:
def remove_extra_newlines_and_sort(input_file_path, output_file_path):
    with open(input_file_path, 'r') as input_file:
        lines = input_file.readlines()

    words = [word.strip() for word in lines if word.strip()]

    sorted_words = sorted(words)

    with open(output_file_path, 'w') as output_file:
        for word in sorted_words:
            output_file.write(word + '\n')

In [27]:
input_file_path = 'words.txt'
output_file_path = 'words_thinned.txt'
remove_extra_newlines_and_sort(input_file_path, output_file_path)

In [28]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

In [29]:
def get_translation(word):
    # word = "wireless"
    url = f"https://www.multitran.com/m.exe?l1=1&l2=2&s={word}"
    response = requests.get(url)
    html_content = response.text

    # Создание объекта Beautiful Soup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Поиск элемента <body> с определенным стилем
    body = soup.body
    container = body.find('div', class_='container')
    mclass_elements = container.find_all(class_='mclass160_10')[2]
    first_table = mclass_elements.find('table')
    rows = first_table.find_all('tr', recursive=False)

    # print(first_table)

    flag = 0

    all_trans = []

    a = 0

    for i, row in enumerate(rows):

        # if i < 3:
        #     continue

        if row.text.strip() == "English thesaurus":
            break

        if row.has_attr('height'):
            a = 2

        # Находим все элементы <td> в текущей строке
        cells = row.find_all('td')
        # Если в строке есть два элемента <td>, то это наши данные subj и trans
        # if len(cells) == 2:
        #     print(len(cells))
        # if not cells:
        #     continue

        # l = len(cells)

        if a > 0:
            flag = 1
            a -= 1
            # print("aaaaaaaa")
        else:
            if len(cells) == 2 and flag > 0:
                trans = cells[1].text.strip()
                trans = trans.split(';')
                trans = [re.sub(r'\s*\([^)]*\)', '', el) for el in trans]
                if len(trans) > 4:
                    trans = trans[:4]
                if len(all_trans) > 10:
                    trans = [trans[0]]
                # print("subj:", subj)
                # print("trans:", trans)
                all_trans.append(trans)
                # print("---------------------")
                flag -= 1


    all_trans = [el for sub in all_trans for el in sub]

    if 'stresses' in all_trans:
        all_trans.remove('stresses')
    return all_trans
    # print(all_trans)

In [30]:
# Открываем файл для чтения
all_res = []
with open("wordsr_thinned.txt", "r") as file:
    # Читаем все содержимое файла
    words = file.read().split()

    # Перебираем слова и вызываем для каждого get_translation
    for word in tqdm(words):
        result = get_translation(word)
        all_res.append([word, result])
        # print(result)

    with open('dictionary.txt', 'w') as file:
        for res in all_res:
            file.write(res[0] + ": " + ', '.join(res[1]) + '.\n')

 78%|█████████████████████████████████████████████████████████████▍                 | 832/1069 [04:07<01:10,  3.37it/s]


AttributeError: 'NoneType' object has no attribute 'find_all'